# Welcome to the Fake News Analyzer!

## First steps

 Here we import all the libraries needed for the project and import the data from the CSV. All the data is in one csv named "fake.csv". This csv has got several rows and columns that will be evaluated in this notebook.

In [1]:
#imports and reading dataset
import pandas as pd
import sklearn as skl
import nltk
import numpy as np

print("-> FAKE NEWS ANALYZER <-")


fake = pd.read_csv("./archive/fake.csv", na_values=[""])
print(fake.head(n=5))


-> FAKE NEWS ANALIZER <-
                                       uuid  ord_in_thread  \
0  6a175f46bcd24d39b3e962ad0f29936721db70db              0   
1  2bdc29d12605ef9cf3f09f9875040a7113be5d5b              0   
2  c70e149fdd53de5e61c29281100b9de0ed268bc3              0   
3  7cf7c15731ac2a116dd7f629bd57ea468ed70284              0   
4  0206b54719c7e241ffe0ad4315b808290dbe6c0f              0   

                 author                      published  \
0     Barracuda Brigade  2016-10-26T21:41:00.000+03:00   
1  reasoning with facts  2016-10-29T08:47:11.259+03:00   
2     Barracuda Brigade  2016-10-31T01:41:49.479+02:00   
3                Fed Up  2016-11-01T05:22:00.000+02:00   
4                Fed Up  2016-11-01T21:56:00.000+02:00   

                                               title  \
0  Muslims BUSTED: They Stole Millions In Gov’t B...   
1  Re: Why Did Attorney General Loretta Lynch Ple...   
2  BREAKING: Weiner Cooperating With FBI On Hilla...   
3  PIN DROP SPEECH BY FATHER 

As we can see it is a pretty big dataset. Let's talk numbers!

In [2]:
print("Number of rows: ", len(fake.index))

print("Number of columns: ", len(fake.columns))

Number of rows:  12999
Number of columns:  20


There are 12999 rows and 20 different columns. This constitutes a big sample of fake news to analyse. Let's now see which type of data we have.

In [3]:
print("-> Printing collumn's data types <-")

print(fake.dtypes)


-> Printing collumn's data types <-
uuid                   object
ord_in_thread           int64
author                 object
published              object
title                  object
text                   object
language               object
crawled                object
site_url               object
country                object
domain_rank           float64
thread_title           object
spam_score            float64
main_img_url           object
replies_count           int64
participants_count      int64
likes                   int64
comments                int64
shares                  int64
type                   object
dtype: object


# Data Cleaning

## Duplicates

Let's start by removing any data duplicates that add nothing to the dataset. We should compare the number of rows before and after removing the duplicates.

In [4]:
print("-> DATA CLEANING <-")

print("-> Dropping duplicate rows <-")

#cleaning duplicates
print("Number of rows before: ", len(fake.index))

fake = fake.drop_duplicates()

print("Number of rows after: ", len(fake.index))


-> DATA CLEANING <-
-> Dropping duplicate rows <-
Number of rows before:  12999
Number of rows after:  12999


So we can see that there are no duplicate rows. Moving on to missing data.

## Missing data

We should find all the rows with missing data and acknowledge every missing information column-wise. Therefore, we should see check for each column the missing information and see if we should either remove the collumn totally or find a viable substitution for each column.

In [5]:
print("-> Showing null values per collumn - before cleaning <-")

print(fake.isnull().sum().sort_values(ascending=False))


-> Showing null values per collumn - before cleaning <-
domain_rank           4223
main_img_url          3643
author                2424
title                  680
country                176
text                    46
thread_title            12
shares                   0
comments                 0
likes                    0
participants_count       0
replies_count            0
uuid                     0
spam_score               0
ord_in_thread            0
site_url                 0
crawled                  0
language                 0
published                0
type                     0
dtype: int64


Here we can see a descending order of the number of missing values per collumn. We will now analyse each column and study if it is worth "fixing" or substituting the missing values or just delete the column all together.

As we can see, we have a column named "main_img_url" that doesn't provide any useful data for the study of this dataset. Because of that, we decided to remove it all together.

In [6]:
print("-> Dropping 'main_img_url' collumn <-")

fake.drop(['main_img_url'], axis=1, inplace=True)


-> Dropping 'main_img_url' collumn <-


In the column "author" there are two different cases that caught our attention: there are "anonymous" authors and just missing authors. Since both of these cases are comparable, because there is no info about the author in neither of them, we decided to make them all the same and add "anonymous" to the rows where the "author" info is missing.

In [7]:
print("-> Fixing null values in 'author' <-")

fake["author"]=fake["author"].fillna("Anonymous")


-> Fixing null values in 'author' <-


As we can also see, there are 12 rows without a thread title. This doesn't allow us group up the fake news into threads because we don't know where they belong. Since it is such a small ammount of news (12) we decided that they should removed as they don't represent a big sample.

In [8]:
print("-> Fixing null values in 'thread_title' <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['thread_title'].notna()]

print("Number of rows after: ", len(fake.index))


-> Fixing null values in 'thread_title' <-
Number of rows before:  12999
Number of rows after:  12987


Since we are going to evaluate and perform text search based on the text of the news, news with no text become irrelevant to this dataset. Therefore, we are going to remove those rows as well.

In [9]:
print("-> Fixing null values in 'text' <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['text'].notna()]

print("Number of rows after: ", len(fake.index))


-> Fixing null values in 'text' <-
Number of rows before:  12987
Number of rows after:  12941


There are news with no title which we have found to be in the same thread where only the news in first place (order_in_thread=0) has got a title. Let's verify if all the existing titles are the same as the thread_title associated.

In [10]:
# get unique values from each column
titles = fake['title'].unique()
thread_titles = fake['thread_title'].unique()

#check if size is the same
print("Number of unique titles:", len(titles))
print("Number of unique thread_titles:", len(thread_titles),"\n")

# convert every value to string
for i in range(0, len(titles)):
    titles[i]=str(titles[i])

for i in range(0, len(thread_titles)):
    thread_titles[i] = str(thread_titles[i])

#sort arrays
np.sort(titles)
np.sort(thread_titles)


# print every different title - compare to bigger list: thread_titles
for element in thread_titles:
    if(element not in titles):
        print("Different thread_title:", element)



Number of unique titles: 11653
Number of unique thread_titles: 11742 

Different thread_title: Did Google Kill Julian Assange?
Different thread_title: Prominent Democrat Connected To Clintons Donated $675,000 To Campaign Of Deputy FBI Director's Wife
Different thread_title: Is it coming into clearer focus for Americans, or not?
Different thread_title: Weiner Sings Like a Bird as Dickileak Provides Exit as Huma Sequestered by FBI at MOUNT WEATHER - After Huma's 65,000 Emails Found...No Longer Together
Different thread_title: Installing a President by Force: Hillary Clinton and Our Moribund Democracy
Different thread_title: Clinton's Policies Look Like a Death Sentence for Americans - American Herald Tribune
Different thread_title: is bed the EU made, now it must lie in it -
Different thread_title: Justin Trudeau Doesn't Tell the Truth to Canadians about Syria
Different thread_title: 10,000 children are missing and the world just looks away
Different thread_title: The Orlando Shooting an

As we can see, there are different titles and thread_titles, making it impossible for us to eliminate one of those columns. We can, however, substitute the data and eliminate rows which don't have any of these informations, as it is impossible for us to track the news associated to any theme or key word.

In [11]:
print("Number of rows with no title and no thread_title:", len(fake.loc[fake['title'].isna() & fake['thread_title'].isna()]))

Number of rows with no title and no thread_title: 0


Since there are no rows with neither of the values in the referred columns, we should turn our attention to how to correct the missing data in these columns. For every row with no title, we will substitute the value with the thread_title instead, since the thread_title will be in some way connected to the theme of the news.

In [12]:
print("-> Fixing null values in 'title' <-")

fake.title.fillna(fake.thread_title, inplace=True)


-> Fixing null values in 'title' <-


Moreover, there are news with no 'country' value associated, more specifically, 176 news. To make it easier to analyze this data and not have missing values, we will drop these rows, since we can't track the origin of the news and it is a rather small ammount of data that is discarded (176 rows), given the size of the dataset.

In [13]:
print("-> Fixing null values in 'country' <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['country'].notna()]

print("Number of rows after: ", len(fake.index))


-> Fixing null values in 'country' <-
Number of rows before:  12941
Number of rows after:  12765


As domain rank is a metric, going from 0 to 100, the later being the strongest, evaluated based on many factors dependent on user searches and the authority of a certain domain, if a domain doesn't have an available ranking, we can suppose it is 0 since it hasn't been evaluated yet. That can be due to lack of information or visits to that domain. That being said, we will substitute every missing value in the column "domain_rank" by 0.

In [14]:
print("-> Fixing null values in 'domain_rank' <-")

fake["domain_rank"] = fake["domain_rank"].fillna(0)


-> Fixing null values in 'domain_rank' <-


Let's take a new look at the null values per column and see the final dataset, with no missing data.

In [15]:
print("-> Showing null values per collumn - after cleaning <-")

print(fake.isnull().sum().sort_values(ascending=False))


-> Showing null values per collumn - after cleaning <-
uuid                  0
domain_rank           0
shares                0
comments              0
likes                 0
participants_count    0
replies_count         0
spam_score            0
thread_title          0
country               0
ord_in_thread         0
site_url              0
crawled               0
language              0
text                  0
title                 0
published             0
author                0
type                  0
dtype: int64


Lastly, we have observed that uuid column doesn't really add anything of value to our dataset, therefore, we will drop the column all together before moving on.

In [16]:
print("-> Dropping 'uuid' collumn <-")

fake.drop(['uuid'], axis=1, inplace=True)


-> Dropping 'uuid' collumn <-


## Broken Data & Data Types

As in all datasets, there are rows with broken data, as in, data that was unsuccessfully crawled and filled in rows with broken values. Let's find these rows in our dataset and correct or drop them.

The first case we noticed were some rows filled with 0 and 1, namely in the title and text columns. Let's locate them and print them, to analize the data and see if it should be dropped. If we can't find any and since we already cleaned some of the data, it means they were already dropped in the steps before.


In [17]:
print("-> Cleaning crawling errors <-")

print(fake.loc[fake['title'] == "0"])
print(fake.loc[fake['title'] == "1"])

print(fake.loc[fake['text'] == "0"])
print(fake.loc[fake['text'] == "1"])


-> Cleaning crawling errors <-
Empty DataFrame
Columns: [ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, replies_count, participants_count, likes, comments, shares, type]
Index: []
Empty DataFrame
Columns: [ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, replies_count, participants_count, likes, comments, shares, type]
Index: []
Empty DataFrame
Columns: [ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, replies_count, participants_count, likes, comments, shares, type]
Index: []
Empty DataFrame
Columns: [ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, replies_count, participants_count, likes, comments, shares, type]
Index: []


As an additional verification that the above statement was correct, we opened the dataset in excel and verified that every case of broken data was indeed gone from the dataset at this point of the cleaning.

The other big case for broken data that we found were some news that weren't correctly crawled and just extended the text of the news throughout several columns. There are several ways of correcting this issue, but in order to get all the cases and clean the data the most, we will start by verifying column by column if the data types and formats are correct, dropping the columns in which that isn't verified.

Starting with 'ord_in_thread' column. It must be a number, therefore let's eliminate every row that doesn't just contain a number. If there is a row with alpha characters in it, the type of the column won't be int64 so let's first verify the type and eliminate non numeric rows if it isn't 'int64'.

In [18]:
print("-> Cleaning 'ord_in_thread' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'ord_in_thread':", fake['ord_in_thread'].dtypes)
print("Is it int64?", fake['ord_in_thread'].dtypes == 'int64')

print("Number of rows after: ", len(fake.index))


-> Cleaning 'ord_in_thread' collumn <-
Number of rows before:  12765
Type of 'ord_in_thread': int64
Is it int64? True
Number of rows after:  12765


As it returns 'int64' we are sure that the whole column only contains numeric values. Moving on to 'author', there are 2 errors to fix: wrong datatypes and the "-NO AUTHOR-" rows. As it is very hard to verify the correctness of the author name, as it is very hard to define "correctness" in these cases, and not exclude valid names with numeric digits using the usual rules for identifying names (since authors can have numbers in their username), we will move on to correcting the "-NO AUTHOR-" cases, where it should be replaced by "Anonymous", as we did before with null values. We will also remove rows with just whitespaces.

In [19]:
print("-> Cleaning 'author' collumn <-")

print("Number of rows before: ", len(fake.index))

fake["author"] = fake["author"].replace("-NO AUTHOR-","Anonymous")

# clean string rows with just whitespace
fake = fake[~fake['author'].str.contains(r'^\s*$')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'author' collumn <-
Number of rows before:  12765
Number of rows after:  12765


Next up we have the column "published", that should only contain dates. We can verify this through a regex expression that verifies the format of the dates, as well as the datatypes present.

In [20]:
print("-> Cleaning 'published' collumn <-")

print("Number of rows before: ", len(fake.index))

fake=fake[fake['published'].str.contains('\d{4}-\d{2}-\d{2}')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'published' collumn <-
Number of rows before:  12765
Number of rows after:  12765


The next column to verify is the column "title". We have cleaned this column before so some work was already done. We should just verify that the titles aren't mere strings of whitespaces, as it is extremely difficult to define a "correct" title or text in any way.

In [21]:
print("-> Cleaning 'title' collumn <-")

print("Number of rows before: ", len(fake.index))

# clean string rows with just whitespace
fake = fake[~fake['title'].str.contains(r'^\s*$')]


print("Number of rows after: ", len(fake.index))


-> Cleaning 'title' collumn <-
Number of rows before:  12765
Number of rows after:  12765


Moving on to the "text" column, we must do the same verification as in the "title" column.

In [22]:
print("-> Cleaning 'text' collumn <-")

print("Number of rows before: ", len(fake.index))

fake = fake[~fake['text'].str.contains(r'^\s*$')]  #clean string rows with just whitespace

print("Number of rows after: ", len(fake.index))


-> Cleaning 'text' collumn <-
Number of rows before:  12765
Number of rows after:  12657


Next up we have the "language" column. This column must only contain one word and no digits, so the funtion isalpha() will do the job.

In [23]:
print("-> Cleaning 'language' collumn <-")

print("Number of rows before: ", len(fake.index))

fake['language'] = fake['language'].apply(str)
fake = fake[fake.language.str.isalpha()] 

# clean string rows with just whitespace
fake = fake[~fake['language'].str.contains(r'^\s*$')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'language' collumn <-
Number of rows before:  12657
Number of rows after:  12657


Next up is the column "crawled", which should only contain the date in which the data was crawled.

In [24]:
print("-> Cleaning 'crawled' collumn <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['crawled'].str.contains(r'\d{4}-\d{2}-\d{2}')]  # dates

print("Number of rows after: ", len(fake.index))


-> Cleaning 'crawled' collumn <-
Number of rows before:  12657
Number of rows after:  12657


The following column is "site_url". Every site follows the same structure of domain.xxx, so we will apply a regex expression to filter unwanted formats.

In [25]:
print("-> Cleaning 'site_url' collumn <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['site_url'].str.contains(r'^(www\.)?(.*)?\/?(.)*')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'site_url' collumn <-
Number of rows before:  12657
Number of rows after:  12657


C:\Users\shit\AppData\Local\Temp/ipykernel_9692/1779741962.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  fake = fake[fake['site_url'].str.contains(r'^(www\.)?(.*)?\/?(.)*')]


Moving on to the 'country' column, the countries are mentioned as a 2 uppercase letter code. Let's confirm every row follows that format.

In [26]:
print("-> Cleaning 'country' collumn <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['country'].str.contains(r'[A-Z]{2}')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'country' collumn <-
Number of rows before:  12657
Number of rows after:  12657


Next up is the column 'domain_rank', which should be a number. Let's verify that as well.

In [27]:
print("-> Cleaning 'domain_rank' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'domain_rank':", fake['domain_rank'].dtypes)
print("Is it int64?", fake['domain_rank'].dtypes=='int64')

#cast to int64
fake['domain_rank'] = fake['domain_rank'].apply(np.int64)

#check if all values are greater or equal than 0
fake=fake[fake['domain_rank']>=0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'domain_rank' collumn <-
Number of rows before:  12657
Type of 'domain_rank': float64
Is it int64? False
Number of rows after:  12657


Next up is "thread_title". This column follows the same rules as the 'title' column, it is very hard to define what is a correct title or not. The only verifiable condition is the same as the one used before for 'title'.

In [28]:
print("-> Cleaning 'thread_title' collumn <-")

print("Number of rows before: ", len(fake.index))

# clean string rows with just whitespace
fake = fake[~fake['thread_title'].str.contains(r'^\s*$')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'thread_title' collumn <-
Number of rows before:  12657
Number of rows after:  12657


The next column is "spam_score". This is a very unique column as it only must have float values between 0 and 1. Let's verify this.

In [29]:
print("-> Cleaning 'spam_score' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'spam_score':", fake['spam_score'].dtypes)
print("Is it float64?", fake['spam_score'].dtypes == 'float64')

#check if all values are greater or equal than 0 and lower or equal than 1
fake = fake[fake['spam_score'] >= 0]
fake = fake[fake['spam_score'] <= 1]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'spam_score' collumn <-
Number of rows before:  12657
Type of 'spam_score': float64
Is it float64? True
Number of rows after:  12657


The next column is "replies_count" which, as a counter, should be a positive integer or zero. Let's check those conditions.

In [30]:
print("-> Cleaning 'replies_count' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'replies_count':", fake['replies_count'].dtypes)
print("Is it int64?", fake['replies_count'].dtypes == 'int64')

#check if all values are greater or equal than 0
fake = fake[fake['replies_count'] >= 0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'replies_count' collumn <-
Number of rows before:  12657
Type of 'replies_count': int64
Is it int64? True
Number of rows after:  12657


Same verification as "replies_count" will happen for the columns "participants_count", "likes", "comments" and "shares". 

In [31]:
print("-> Cleaning 'participants_count' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'participants_count':", fake['participants_count'].dtypes)
print("Is it int64?", fake['participants_count'].dtypes == 'int64')

#check if all values are greater or equal than 0
fake = fake[fake['participants_count'] >= 0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'participants_count' collumn <-
Number of rows before:  12657
Type of 'participants_count': int64
Is it int64? True
Number of rows after:  12657


In [32]:
print("-> Cleaning 'likes' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'likes':", fake['likes'].dtypes)
print("Is it int64?", fake['likes'].dtypes == 'int64')

#check if all values are greater or equal than 0
fake = fake[fake['likes'] >= 0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'likes' collumn <-
Number of rows before:  12657
Type of 'likes': int64
Is it int64? True
Number of rows after:  12657


In [33]:
print("-> Cleaning 'comments' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'comments':", fake['comments'].dtypes)
print("Is it int64?", fake['comments'].dtypes == 'int64')

#check if all values are greater or equal than 0
fake = fake[fake['comments'] >= 0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'comments' collumn <-
Number of rows before:  12657
Type of 'comments': int64
Is it int64? True
Number of rows after:  12657


In [34]:
print("-> Cleaning 'shares' collumn <-")

print("Number of rows before: ", len(fake.index))

print("Type of 'shares':", fake['shares'].dtypes)
print("Is it int64?", fake['shares'].dtypes == 'int64')

#check if all values are greater or equal than 0
fake = fake[fake['shares'] >= 0]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'shares' collumn <-
Number of rows before:  12657
Type of 'shares': int64
Is it int64? True
Number of rows after:  12657


Last but not least we have the "type" column, which contains a lowercase string classifying the news.

In [35]:
print("-> Cleaning 'type' collumn <-")

print("Number of rows before: ", len(fake.index))

fake = fake[fake['type'].str.contains(r'^[a-z]+$')]  # lowercase letters

# clean string rows with just whitespace
fake = fake[~fake['type'].str.contains(r'^\s*$')]

print("Number of rows after: ", len(fake.index))


-> Cleaning 'type' collumn <-
Number of rows before:  12657
Number of rows after:  12657


And now, since the data cleaning is finished, we can save the new clean data to a new csv file, to be used later.

In [36]:
fake.to_csv("fake_clean.csv", index=False)